## Assignment 1 (40 marks)
#### =====================================================================================================
### Deadline: 09/08 11:59 pm
#### =====================================================================================================

### Problem 1: SVM Classification (30 marks)

`lab01_dataset_1.xlsx` contains the claim history of 27,513 homeowner policies. The following table describes the eleven columns in the dataset.

| Name | Description |
| --- | --- |
| policy | Policy Identifier |
| exposure | Duration a Policy exposed in a Year |
| num_claims | Number of Claims in a Year |
| amt_claims | Total Claim Amount in a Year	|
| f_primary_age_tier | Age Tier of Primary Insured |
| f_primary_gender | Gender of Primary Insured |
| f_marital | Marital Status of Primary Insured |
| f_residence_location | Location of Residence Property |
| f_fire_alarm_type | Fire Alarm Type |
| f_mile_fire_station | Distance to Nearest Fire Station |
| f_aoi_tier | Amount of Insurance Tier |

We want to predict the *Frequency* which is the *number of claims per unit of exposure* using the above features.  We first divide the reported number of claims by the exposure. This gives us the *Frequency*.  Next, we put the policies into five groups according to their *Frequency* values. We will use this *Group* as our target variable which has five classes.

| Group | Values |
| :--- | :--- |
| 0 | Frequency = 0 |
| 1 | 0 < Frequency <= 1 |
| 2 | 1 < Frequency <= 2 |
| 3 | 2 < Frequency <= 3 |
| 4 | 3 < Frequency |


### 1.a (5 marks)
Create a new column for the dataset which will indiciate the *Frequency Group* and output the updated dataset.

In [50]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, OneHotEncoder
data = pd.read_excel('lab01_dataset_1-1.xlsx')
data.head()

,policy,exposure,num_claims,amt_claims,f_primary_age_tier,f_primary_gender,f_marital,f_residence_location,f_fire_alarm_type,f_mile_fire_station,f_aoi_tier
0,P00001,1.0,0,0.00,21 - 27,Male,Married,Urban,Alarm Service,< 1 mile,351K - 600K
1,G00002,1.0,0,0.00,38 - 60,Male,Un-Married,Suburban,NaN,1 - 5 miles,< 100K
2,A00003,1.0,2,3079.01,38 - 60,Female,Married,Suburban,Standalone,< 1 mile,100K - 350K
3,P00004,1.0,1,804.87,28 - 37,Female,Un-Married,Suburban,Standalone,1 - 5 miles,< 100K
4,G00005,1.0,1,638.74,28 - 37,Female,Un-Married,Suburban,Alarm Service,< 1 mile,100K - 350K


In [51]:
def categorizeFrequency(frequency):
    if frequency == 0:
        return 0
    elif 0 < frequency <= 1:
        return 1
    elif 1 < frequency <= 2:
        return 2
    elif 2 < frequency <= 3:
        return 3
    else:
        return 4
    
data['Frequency'] = data['num_claims'] / data['exposure'] 
data['Frequency Group'] = data['Frequency'].apply(categorizeFrequency)

In [52]:
data.head()

,policy,exposure,num_claims,amt_claims,f_primary_age_tier,f_primary_gender,f_marital,f_residence_location,f_fire_alarm_type,f_mile_fire_station,f_aoi_tier,Frequency,Frequency Group
0,P00001,1.0,0,0.00,21 - 27,Male,Married,Urban,Alarm Service,< 1 mile,351K - 600K,0.0,0
1,G00002,1.0,0,0.00,38 - 60,Male,Un-Married,Suburban,NaN,1 - 5 miles,< 100K,0.0,0
2,A00003,1.0,2,3079.01,38 - 60,Female,Married,Suburban,Standalone,< 1 mile,100K - 350K,2.0,2
3,P00004,1.0,1,804.87,28 - 37,Female,Un-Married,Suburban,Standalone,1 - 5 miles,< 100K,1.0,1
4,G00005,1.0,1,638.74,28 - 37,Female,Un-Married,Suburban,Alarm Service,< 1 mile,100K - 350K,1.0,1


### 1.b (5 marks)
There are seven categorial features in the dataset namely, *f_aoi_tier, f_primary_age_tier, f_fire_alarm_type, f_marital, f_mile_fire_station, f_primary_gender, f_residence_location*. Display all the unique values of these seven categories.

In [53]:
featureList = ['f_aoi_tier', 'f_primary_age_tier', 'f_fire_alarm_type', 'f_marital', 'f_mile_fire_station', 'f_primary_gender', 'f_residence_location']
for feature in featureList:
    print(list(data[feature].unique()))

['351K - 600K', '< 100K', '100K - 350K', '601K - 1M', '> 1M']
['21 - 27', '38 - 60', '28 - 37', '> 60', '< 21']
['Alarm Service', nan, 'Standalone']
['Married', 'Un-Married', 'Not Married']
['< 1 mile', '1 - 5 miles', '> 10 miles', '6 - 10 miles']
['Male', 'Female']
['Urban', 'Suburban', 'Rural']


### 1.c (5 marks)
We will train SVM models using those seven categorical features. However, their values are currently all categorical data, but SVM requires them to be numerical. Perform ***one-hot encoding*** on these features to obtain an updated dataset which has only numerical values.

In [57]:
oneHotEncoder = OneHotEncoder(handle_unknown = 'ignore', sparse_output = False)
oneHotEncoder.set_output(transform = 'pandas')
encodedDf = oneHotEncoder.fit_transform(data[featureList])
print(encodedDf.columns)
encodedDf.head()

Index(['f_aoi_tier_100K - 350K', 'f_aoi_tier_351K - 600K',
       'f_aoi_tier_601K - 1M', 'f_aoi_tier_< 100K', 'f_aoi_tier_> 1M',
       'f_primary_age_tier_21 - 27', 'f_primary_age_tier_28 - 37',
       'f_primary_age_tier_38 - 60', 'f_primary_age_tier_< 21',
       'f_primary_age_tier_> 60', 'f_fire_alarm_type_Alarm Service',
       'f_fire_alarm_type_Standalone', 'f_fire_alarm_type_nan',
       'f_marital_0.0', 'f_marital_1.0', 'f_marital_2.0',
       'f_mile_fire_station_1 - 5 miles', 'f_mile_fire_station_6 - 10 miles',
       'f_mile_fire_station_< 1 mile', 'f_mile_fire_station_> 10 miles',
       'f_primary_gender_Female', 'f_primary_gender_Male',
       'f_residence_location_Rural', 'f_residence_location_Suburban',
       'f_residence_location_Urban'],
      dtype='object')


,f_aoi_tier_100K - 350K,f_aoi_tier_351K - 600K,f_aoi_tier_601K - 1M,f_aoi_tier_< 100K,f_aoi_tier_> 1M,f_primary_age_tier_21 - 27,f_primary_age_tier_28 - 37,f_primary_age_tier_38 - 60,f_primary_age_tier_< 21,f_primary_age_tier_> 60,...,f_marital_2.0,f_mile_fire_station_1 - 5 miles,f_mile_fire_station_6 - 10 miles,f_mile_fire_station_< 1 mile,f_mile_fire_station_> 10 miles,f_primary_gender_Female,f_primary_gender_Male,f_residence_location_Rural,f_residence_location_Suburban,f_residence_location_Urban
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0


In [55]:
moodUniqueValues = list(np.asarray(data['f_marital'].unique()))
print(moodUniqueValues)
encoderMood = OrdinalEncoder(categories=[list(moodUniqueValues)])
data['f_marital'] = encoderMood.fit_transform(data[['f_marital']])
data.head()

['Married', 'Un-Married', 'Not Married']


,policy,exposure,num_claims,amt_claims,f_primary_age_tier,f_primary_gender,f_marital,f_residence_location,f_fire_alarm_type,f_mile_fire_station,f_aoi_tier,Frequency,Frequency Group
0,P00001,1.0,0,0.00,21 - 27,Male,0.0,Urban,Alarm Service,< 1 mile,351K - 600K,0.0,0
1,G00002,1.0,0,0.00,38 - 60,Male,1.0,Suburban,NaN,1 - 5 miles,< 100K,0.0,0
2,A00003,1.0,2,3079.01,38 - 60,Female,0.0,Suburban,Standalone,< 1 mile,100K - 350K,2.0,2
3,P00004,1.0,1,804.87,28 - 37,Female,1.0,Suburban,Standalone,1 - 5 miles,< 100K,1.0,1
4,G00005,1.0,1,638.74,28 - 37,Female,1.0,Suburban,Alarm Service,< 1 mile,100K - 350K,1.0,1


### 1.d (5 marks)
Divide the observations into training and testing partitions. Observations whose *Policy Identifier* starts with the letters A, G, and P will go to the training partition. The remaining observations go to the testing partition. Output the total number of policies present in the Training partition and Testing partition.

In [ ]:

A, G, and P will go to the training partition
remaining testing


### 1.e (5 marks)
Train an SVM model using [LinearSVR](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVR.html). The input features will be the encoded version of the feature set *f_aoi_tier, f_primary_age_tier, f_fire_alarm_type, f_marital, f_mile_fire_station, f_primary_gender, f_residence_location* and the output is the *Frequency Group*. Use `verbose=1` to observe the optimization steps during the training process.

### 1.f (5 marks)
Compute and output the Accuracy Score on the Testing partition.

### Problem 2: Perceptron Learning (10 marks)

The dataset `lab01_dataset_2.csv` has a *3-dimensional input space* and a class label of *Positive* and *Negative*. For this task, you are **not allowed** to use any *functionalities* of the `sklearn` module.

### 2.a (5 marks)
Write a function `my_perceptron()` which applies the perceptron algorithm (refer to the lecture slide covering linear separators for details of this algorithm) on the dataset to create a linear separator. `my_perceptron()` takes the dataset as its input and returns a ***3-dimensional weight vector*** which can be used to create the linear separator (assume `bias = 0`). Use the *initial weights* `w = [1,2,3]`. Use a classification threshold of `99%` i.e., `my_perceptron()` will terminate once the misclassification rate is less than `1%`.

### 2.b (5 marks)
Create a *3D plot* which showcases the dataset in a 3D-space alongwith the *linear separator* you obtained from `my_perceptron()`. Use two different colors to represent the data points belonging in the two classes for ease of viewing.